In [ ]:
%load_ext autoreload
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib.backends.backend_pdf import PdfPages
from scipy.integrate import simpson
from kernels import IonProb, IonRate, analyticalRate
from field_functions import LaserField
from __init__ import FourierTransform

class AU:
    meter = 5.2917720859e-11 # atomic unit of length in meters
    nm = 5.2917721e-2 # atomic unit of length in nanometres
    second = 2.418884328e-17 # atomic unit of time in seconds
    fs = 2.418884328e-2 # atomic unit of time in femtoseconds
    Joule = 4.359743935e-18 # atomic unit of energy in Joules
    eV = 27.21138383 # atomic unit of energy in electronvolts
    Volts_per_meter = 5.142206313e+11 # atomic unit of electric field in V/m
    Volts_per_Angstrom = 51.42206313 # atomic unit of electric field in V/Angström
    speed_of_light = 137.035999 # vacuum speed of light in atomic units
    Coulomb = 1.60217646e-19 # atomic unit of electric charge in Coulombs
    PW_per_cm2_au = 0.02849451308 # PW/cm^2 in atomic units
AtomicUnits=AU


file_params = [
    ("850nm_1e+14", 850, 1e14, 350, 1e10, 1, 0, 0),
    #("850nm_1e+14", 850, 1e14, 350, 1e10, 1, 0, -np.pi/2),
    # ("850nm_350nm_7.5e+13", 850, 7.50e13, 350, 6.00e09, 0.93, 0, -np.pi/2),
    # ("900nm_320nm_5e+14", 900, 5e14, 320, 4e10, 0.75, 0, -np.pi/2),
    # ("1200nm_320nm_1e+14", 1200, 1e14, 320, 4e10, 0.75, 0, -np.pi/2),
    # ("900nm_250nm_8e+13", 900, 8e13, 250, 6e8, 0.58, 0, -np.pi/2),
    # ("900nm_250nm_9e+13", 900, 9e13, 250, 6e8, 0.58, 0, -np.pi/2),
    # ("900nm_250nm_1e+14", 900, 1e14, 250, 6e8, 0.58, 0, -np.pi/2),
    # ("900nm_250nm_1.1e+14", 900, 1.1e14, 250, 6e8, 0.58, 0, -np.pi/2),
]

params = {'E_g': 0.5, 'αPol': 4.51, "div_p":2**-4*2, "div_theta":1}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%autoreload
for file_name, lam0_pump, I_pump, lam0_probe, I_probe, FWHM_probe, cep_pump, cep_probe in file_params:
    laser_pulses = LaserField(cache_results=True)
    laser_pulses.add_pulse(lam0_pump, I_pump, cep_pump, lam0_pump/ AtomicUnits.nm / AtomicUnits.speed_of_light)
    t_min, t_max = laser_pulses.get_time_interval()
    time_recon= np.arange(int(t_min), int(t_max)+1, 1.)


    ion_na_rate_GASFIR = IonRate(time_recon, laser_pulses, params, dT=0.5, kernel_type='GASFIR')
    laser_pulses.reset()

In [62]:
%autoreload
for file_name, lam0_pump, I_pump, lam0_probe, I_probe, FWHM_probe, cep_pump, cep_probe in file_params:
    laser_pulses = LaserField(cache_results=True)
    laser_pulses.add_pulse(lam0_pump, I_pump, cep_pump, lam0_pump/ AtomicUnits.nm / AtomicUnits.speed_of_light)
    t_min, t_max = laser_pulses.get_time_interval()
    time_recon= np.arange(int(t_min), int(t_max)+1, 1.)

    
    ion_na_rate_SFA = IonRate(time_recon, laser_pulses, params, dT=0.5, kernel_type='exact_SFA')
    laser_pulses.reset()

In [ ]:
%autoreload
for file_name, lam0_pump, I_pump, lam0_probe, I_probe, FWHM_probe, cep_pump, cep_probe in file_params:
    laser_pulses = LaserField(cache_results=True)
    laser_pulses.add_pulse(lam0_pump, I_pump, cep_pump, lam0_pump/ AtomicUnits.nm / AtomicUnits.speed_of_light)
    t_min, t_max = laser_pulses.get_time_interval()
    time_recon= np.arange(int(t_min), int(t_max)+1, 1.)
    # to save computation time we can neglect cross terms!! just look at formula for <p|d|psi> there is i^l and because of the transition rules l has to be +-1 so if we sum over all states and one is complex conjugatet and we sum it up they will cancel out !!! but be carefull we only can use states that are allowed theoretically


    #ion_na_rate_SFA_excited = IonRate(time_recon, laser_pulses, params, dT=0.5, kernel_type='exact_SFA', excitedStates=3, coeffType="trecx") #it uses the wrong coefficients for the calculation, it uses the 2s state for excitedSTates=2 and 2s is not even there
    ion_na_rate_SFA_excited = IonRate(time_recon, laser_pulses, params, dT=0.5, kernel_type='exact_SFA', excitedStates=3, coeffType="numerical")
    #ionprob = IonProb(laser_pulses, params, dT=0.5, kernel_type='exact_SFA', excitedStates=2)
    #ion_na_rate_GASFIR = IonRate(time_recon, laser_pulses, params, dT=0.5, kernel_type='GASFIR')
    laser_pulses.reset()

Basis states (6): [(1, 0, 0), (2, 0, 0), (2, 1, 0), (3, 0, 0), (3, 1, 0), (3, 2, 0)]


In [24]:
#print("ion_prob_GASFIR", 1-np.exp(-np.double(simpson(ion_na_rate_GASFIR, x=time_recon, axis=-1, even='simpson'))))
print("ion_prob_excited", 1-np.exp(-np.double(simpson(np.real(4*np.pi*ion_na_rate_SFA_excited), x=time_recon, axis=-1, even='simpson'))))
print("ion_prob_SFA", 1-np.exp(-np.double(simpson(ion_na_rate_SFA, x=time_recon, axis=-1, even='simpson'))))
print("ion_prob_tRecX", 0.008607)

ion_prob_excited 0.00011684229991948403
ion_prob_SFA 0.000537496672368909
ion_prob_tRecX 0.008607


In [69]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=time_recon, y=np.real(ion_na_rate_SFA_excited), mode='lines', name='SFA_excited')) #they are exactly 4pi apart
fig.add_trace(go.Scatter(x=time_recon, y=ion_na_rate_SFA, mode='lines', name='SFA'))

fig.update_layout(
    title=f'Ionization Rate Comparison for {file_name}',
    xaxis_title='Time (a.u.)',
    yaxis_title='Ionization Rate (a.u.)',
    legend=dict(x=0.01, y=0.99),
    template='plotly_white',
    xaxis_range=[-100, 100],
    width=800,
    height=600
)
# truncate y-axis
fig.show()